In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, element
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

Webscraping:

Create a program that can extract information from housing aggregator websites like gratka.pl or otodom.pl. The program should be able to extract all the relevant data from a search query, such as location, minimum and maximum price, minimum and maximum area, and other important features. It should also include options for private or public ads. The extracted data should include the link to the record, price, number of rooms, area, description, other features, and position on a map. You can extract the position using Google Maps iframes.

 

Once you have extracted the data, use it to gain insights about a smaller town. The program should allow users to input the town they are interested in and then extract all the relevant information about available housing in that area. This will provide useful information for those who are looking to move to a new town or city, or for anyone who is interested in the housing market in a particular area.

In [ ]:
### There is an issue with the code there should be omit allers included

## Gratka.pl

In [34]:
def house(city:str, low_price:int, high_price:int, low_sqm:int, high_sqm:int, num_of_pages:int): 
    ### 1 Step
    # Opening Google
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    # Direct to predefine website
    driver.get('https://gratka.pl/')

    ### 1.5 Step
    # Accespting cookies
    time.sleep(2)
    cookie = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div[6]/button[2]')
    cookie.click()
    time.sleep(1)

    ### 2 Step
    ## Navigate to real-estate
    real_estates = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[2]/ul/li[1]/a')
    real_estates.click()
    # All categories
    all_estates = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[2]/ul/li[2]/ul/li[8]/a')
    all_estates.click()
    # location
    loc_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[1]/input')
    loc_bar.send_keys(city) #variable
    time.sleep(2)
    loc_bar.send_keys(Keys.ENTER)
    ## Radius
    # click radius
    #radius = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[2]/div[2]/div/div/span')
    #radius.click()
    #time.sleep(2)
    # choosing specific radius - 5km
    #radius_km = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[2]/div[2]/div/div/div/ul/li[1]')
    #radius_km.click()
    time.sleep(1)
    ## Price Range
    # From
    price_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[3]/div/div[1]/div/input')
    price_bar.click()
    low_price = int(low_price)
    price_bar.send_keys(low_price)
    # To
    price_to = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[3]/div/div[2]/div/input')
    price_to.click()
    high_price = int(high_price)
    price_to.send_keys(high_price)
    ## Squared Meters
    # From
    squared_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[6]/div/div[1]/div/input')
    squared_bar.click()
    low_sqm = int(low_sqm)
    squared_bar.send_keys(low_sqm)
    # To
    squared_to = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[6]/div/div[2]/div/input')
    squared_to.click()
    high_sqm = int(high_sqm)
    squared_to.send_keys(high_sqm)
    #time.sleep(2)
    ## Search
    search = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[9]/div')
    search.click()
    search.click()
    time.sleep(2)

    ### 4 Iterate through houses
    soup = BeautifulSoup(driver.page_source)
    houses = list()

    # retriving info about number of pages
    max_pages = int(soup.find('div', {'class':'pagination container'}).text.replace('\n','').split('z')[-1])

    if num_of_pages > int(max_pages):
        # in case user input exceeds the real number of pages then max pages are returned 
        for i in range(1,max_pages+1):
            soup = BeautifulSoup(driver.page_source)
            properties = soup.find_all('div', {'class': 'listing__teaserWrapper'})
            for property in properties:

                houses.append({
                        'City': property.find('span', {'class': 'teaserUnified__location'}).text.split(',')[0].replace(' ','').replace('\n',''),
                        'District':  property.find('span', {'class': 'teaserUnified__location'}).text.split(',')[1].replace(' ',''),
                        'Street': property.find('h2', {'class': 'teaserUnified__title'}).text.split(',')[-1].replace('\n',''),
                        'Sq meters': property.find('li', {'class': 'teaserUnified__listItem'}).text,
                        #'Rooms' : (property.find('li', {'class': 'teaserUnified__listItem'})),
                        'Price' : property.find('p', {'class': 'teaserUnified__price'}).text.split('\n')[1],
                        'Price per sq' : property.find('span', {'class': 'teaserUnified__additionalPrice'}).text.split('\n')[1]
                        })
    
            if i == 5:
                # next page
                driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[43]/a[{i}]').click()
            elif i > 5 and i < num_of_pages:
                # next page
                driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[43]/a[7]').click()
            else:
                break

    else:
        for i in range(1,num_of_pages+1):
            soup = BeautifulSoup(driver.page_source)
            properties = soup.find_all('div', {'class': 'listing__teaserWrapper'})
            for property in properties:

                houses.append({
                        'City': property.find('span', {'class': 'teaserUnified__location'}).text.split(',')[0].replace(' ','').replace('\n',''),
                        'District':  property.find('span', {'class': 'teaserUnified__location'}).text.split(',')[1].replace(' ',''),
                        'Street': property.find('h2', {'class': 'teaserUnified__title'}).text.split(',')[-1].replace('\n',''),
                        'Sq meters': property.find('li', {'class': 'teaserUnified__listItem'}).text,
                        #'Rooms' : (property.find('li', {'class': 'teaserUnified__listItem'})),
                        'Price' : property.find('p', {'class': 'teaserUnified__price'}).text.split('\n')[1],
                        'Price per sq' : (property.find('span', {'class': 'teaserUnified__additionalPrice'})).text.split('\n')[1]
                        })
                
            if i <= 5:
                # next page
                driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[43]/a[{i}]').click()
            elif i > 5 and i < num_of_pages:
                # next page
                driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[43]/a[7]').click()
            else:
                break

    df = pd.DataFrame(houses)
    driver.quit()
    return df

In [10]:
### 1 Step
# Opening Google
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Direct to predefine website
driver.get('https://gratka.pl/')

### 1.5 Step
# Accespting cookies
time.sleep(2)
cookie = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div[6]/button[2]')
cookie.click()
time.sleep(1)

### 2 Step
## Navigate to real-estate
real_estates = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[2]/ul/li[1]/a')
real_estates.click()
# All categories
all_estates = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[2]/ul/li[2]/ul/li[8]/a')
all_estates.click()
# location
loc_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[1]/input')
loc_bar.send_keys("Kraków") #variable
time.sleep(2)
loc_bar.send_keys(Keys.ENTER)
## Radius
# click radius
#radius = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[2]/div[2]/div/div/span')
#radius.click()
#time.sleep(2)
# choosing specific radius - 5km
#radius_km = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[2]/div[2]/div/div/div/ul/li[1]')
#radius_km.click()
time.sleep(1)
## Price Range
# From
price_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[3]/div/div[1]/div/input')
price_bar.click()
low_price = int("300000")
price_bar.send_keys(low_price)
# To
price_to = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[3]/div/div[2]/div/input')
price_to.click()
high_price = int("800000")
price_to.send_keys(high_price)
## Squared Meters
# From
squared_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[6]/div/div[1]/div/input')
squared_bar.click()
low_sqm = int("50")
squared_bar.send_keys(low_sqm)
# To
squared_to = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[6]/div/div[2]/div/input')
squared_to.click()
high_sqm = int("100")
squared_to.send_keys(high_sqm)
#time.sleep(2)
## Search
search = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[9]/div')
search.click()
time.sleep(2)
search.click()
#time.sleep(2)

### 4 Iterate through houses
soup = BeautifulSoup(driver.page_source)
houses = list()

# retriving info about number of pages
max_pages = int(soup.find('div', {'class':'pagination container'}).text.replace('\n','').split('z')[-1])

num_of_pages = 7

if num_of_pages > int(max_pages):
    # in case user input exceeds the real number of pages then max pages are returned 
    for i in range(1,max_pages+1):
        soup = BeautifulSoup(driver.page_source)
        properties = soup.find_all('div', {'class': 'listing__teaserWrapper'})
        for proper in properties:

            houses.append({
                'City': proper.find('span', {'class': 'teaserUnified__location'}).text.split(',')[0].replace(' ','').replace('\n',''),
                'District':  proper.find('span', {'class': 'teaserUnified__location'}).text.split(',')[1].replace(' ',''),
                'Street': proper.find('h2', {'class': 'teaserUnified__title'}).text.split(',')[-1].replace('\n',''),
                'Sq meters': proper.find('li', {'class': 'teaserUnified__listItem'}).text,
                #'Rooms' : (proper.find('li', {'class': 'teaserUnified__listItem'})),
                'Price' : proper.find('p', {'class': 'teaserUnified__price'}).text.split('\n')[1],
                'Price per sq' : (proper.find('span', {'class': 'teaserUnified__additionalPrice'})).text.split('\n')[1]
                })

        if i == 5:
            # next page
            driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[43]/a[{i}]').click()
        elif i > 5 and i < num_of_pages:
            # next page
            driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[43]/a[7]').click()
        else:
            break

else:
    for i in range(1,num_of_pages+1):
        soup = BeautifulSoup(driver.page_source)
        properties = soup.find_all('div', {'class': 'listing__teaserWrapper'})
        for proper in properties:

            houses.append({
                'City': proper.find('span', {'class': 'teaserUnified__location'}).text.split(',')[0].replace(' ','').replace('\n',''),
                'District':  proper.find('span', {'class': 'teaserUnified__location'}).text.split(',')[1].replace(' ',''),
                'Street': proper.find('h2', {'class': 'teaserUnified__title'}).text.split(',')[-1].replace('\n',''),
                'Sq meters': proper.find('li', {'class': 'teaserUnified__listItem'}).text,
                #'Rooms' : (proper.find('li', {'class': 'teaserUnified__listItem'})),
                'Price' : proper.find('p', {'class': 'teaserUnified__price'}).text.split('\n')[1],
                'Price per sq' : (proper.find('span', {'class': 'teaserUnified__additionalPrice'})).text.split('\n')[1]
                })

        if i <= 5:
            # next page
            driver.find_element(By.XPATH, f'/html/body/div[3]/div[2]/div[1]/div[43]/a[{i}]').click()
        elif i > 5 and i < num_of_pages:
            # next page
            driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[43]/a[7]').click()
        else:
            break

df = pd.DataFrame(houses)
driver.quit()

ElementClickInterceptedException: Message: element click intercepted: Element <a href="https://gratka.pl/nieruchomosci/krakow?page=...&amp;cena-calkowita:min=300000&amp;cena-calkowita:max=800000&amp;powierzchnia-w-m2:min=50&amp;powierzchnia-w-m2:max=100">5</a> is not clickable at point (465, 663). Other element would receive the click: <div role="button" tabindex="0" class="survicate_light_JL8G_ survicate_overlay_stIWL sv__overlay sv__overlay--light" label="hide-overlay"></div>
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x0074DCE3]
	(No symbol) [0x006E39D1]
	(No symbol) [0x005F4DA8]
	(No symbol) [0x006252EA]
	(No symbol) [0x00623BBA]
	(No symbol) [0x00621EFB]
	(No symbol) [0x00621027]
	(No symbol) [0x00618D05]
	(No symbol) [0x0063AECC]
	(No symbol) [0x00618726]
	(No symbol) [0x0063B224]
	(No symbol) [0x0064D57C]
	(No symbol) [0x0063ACC6]
	(No symbol) [0x00616F68]
	(No symbol) [0x006180CD]
	GetHandleVerifier [0x009C3832+2506274]
	GetHandleVerifier [0x009

In [35]:
huehue = house("Kraków",300000,800000,50,100,7)

ElementClickInterceptedException: Message: element click intercepted: Element <a href="https://gratka.pl/nieruchomosci/krakow?page=...&amp;cena-calkowita:min=300000&amp;cena-calkowita:max=800000&amp;powierzchnia-w-m2:min=50&amp;powierzchnia-w-m2:max=100">5</a> is not clickable at point (465, 663). Other element would receive the click: <div role="button" tabindex="0" class="survicate_light_JL8G_ survicate_overlay_stIWL sv__overlay sv__overlay--light" label="hide-overlay"></div>
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x0074DCE3]
	(No symbol) [0x006E39D1]
	(No symbol) [0x005F4DA8]
	(No symbol) [0x006252EA]
	(No symbol) [0x00623BBA]
	(No symbol) [0x00621EFB]
	(No symbol) [0x00621027]
	(No symbol) [0x00618D05]
	(No symbol) [0x0063AECC]
	(No symbol) [0x00618726]
	(No symbol) [0x0063B224]
	(No symbol) [0x0064D57C]
	(No symbol) [0x0063ACC6]
	(No symbol) [0x00616F68]
	(No symbol) [0x006180CD]
	GetHandleVerifier [0x009C3832+2506274]
	GetHandleVerifier [0x009